In [1]:
# 12. implement Batch Gradient Descent with early stopping for Softmax Regression without using sklearn

In [2]:
import numpy as np
import math

rng = np.random.default_rng(42)

In [3]:
def get_random_sample_indices_no_replacement(total_size, sample_size):
    all_indices = np.arange(total_size)
    rng.shuffle(all_indices)
    sample_indices = all_indices[:sample_size]
    other_indices = all_indices[sample_size:]
    return sample_indices, other_indices

In [4]:
def get_random_sample_indices_replacement(total_size, sample_size):
    sample_indices = rng.integers(0, total_size, sample_size)
    return sample_indices

In [5]:
def compute_softmax_estimated_probabilities(features, parameter_matrix):
    softmax_scores = features.dot(parameter_matrix.T)
    exponentiated_scores = np.exp(softmax_scores)
    
    total_exp_score_by_class = np.apply_along_axis(np.sum, 1, exponentiated_scores)
    
    estimated_probabilities = exponentiated_scores / total_exp_score_by_class[:, None]
    return estimated_probabilities

In [6]:
def compute_cross_entropy_cost(labels, estimated_probabilities):
    m = np.shape(labels)[0]
    logged_estimated_probabilities = np.log(estimated_probabilities)
    cost_matrix = np.multiply(labels, logged_estimated_probabilities)
    cost = (-1/m) * np.sum(cost_matrix)
    return cost

In [7]:
# compute gradient matrix
# it is K rows by n columns, where K is the number of classes and n is the number of features

# features is m rows by n columns, where m is the number of instances
# labels is m rows by K columns, where K is the number of classes
# estimated_probabilities is m rows by K columns

# note that the gradient matrix is 1/m times the transpose of probabilities_minus_labels dotted with features

In [8]:
def compute_gradient_matrix(features, labels, estimated_probabilities):
    m = np.shape(features)[0]
    
    probabilities_minus_labels = estimated_probabilities - labels
    
    gradient_matrix = (1/m) * probabilities_minus_labels.T.dot(features)
    return gradient_matrix

In [9]:
def compute_learning_rate(t, eta0=.01, power_t=.25):
    eta = eta0 / pow(t, power_t)
    return eta    

In [10]:
# method parameter for batch, mini batch, or stochastic gradient calculation
# mini_batch_size parameter for size of mini batch (defaults to 1 for stochastic)

# parameter for number of epochs
# to wait and see if best cost function score improves
# note that an epoch is either a single batch gradient step, single minibatch gradient step, or group m stochastic gradient steps

# parameter for tolerance (only update best cost, best)

# returns best parameter matrix, best cost, and number of gradient descent steps performed (t)

In [11]:
def softmax_regression_with_early_stopping(features, labels, validation_set_ratio=.1, method='stochastic', mini_batch_size=10, n_epochs_to_wait=10_000, tol=0.001):    
    # split into training and validation set
    M = np.shape(features)[0]
    val_set_size = math.ceil(M * validation_set_ratio)
    
    val_indices, train_indices = get_random_sample_indices_no_replacement(M, val_set_size)
    
    train_features = features[train_indices]
    train_labels = labels[train_indices]
    
    val_features = features[val_indices]
    val_labels = labels[val_indices]
    
    # initialize variables and parameter matrix
    m = np.shape(train_features)[0]
    n = np.shape(train_features)[1]
    K = np.shape(train_labels)[1]
    
    parameter_matrix = np.zeros((K, n))
    
    best_cost = np.inf
    best_parameter_matrix = parameter_matrix
    n_epochs_waited = 0
    
    t = 0
    
    tol_t = None
    stochastic_array = None
    stochastic_loop_index = None
    
    # main loop
    while True:
        # get estimated probabilities for validation set
        val_estimated_probabilities = compute_softmax_estimated_probabilities(val_features, parameter_matrix)

        # get cost
        cost = compute_cross_entropy_cost(val_labels, val_estimated_probabilities)

        # compare cost to best_cost for batch and mini batch
        if method == 'batch' or method == 'mini_batch':
            if best_cost - cost > tol:
                best_parameter_matrix = parameter_matrix
                best_cost = cost
                n_epochs_waited = 0
            elif 0 < best_cost - cost:
                best_parameter_matrix = parameter_matrix
                best_cost = cost
                n_epochs_waited += 1
            else:
                n_epochs_waited += 1
                
        # compare cost to best_cost for stochastic 
        elif method =='stochastic':
            if best_cost - cost > tol:
                best_parameter_matrix = parameter_matrix
                best_cost = cost
                tol_t = t
            elif cost < best_cost:
                best_parameter_matrix = parameter_matrix
                best_cost = cost
            
            n_epochs_waited = math.floor((t - tol_t) / m)
            

        # break training if necessary
        if n_epochs_waited >= n_epochs_to_wait:
            return best_parameter_matrix, best_cost, t

        # calculate gradient
        if method == 'batch':
            train_estimated_probabilities = compute_softmax_estimated_probabilities(train_features, parameter_matrix)
            gradient_matrix = compute_gradient_matrix(train_features, train_labels, train_estimated_probabilities)
        elif method == 'mini_batch':
            mini_batch_indices = get_random_sample_indices_replacement(m, mini_batch_size)
            mini_batch_estimated_probabilities = compute_softmax_estimated_probabilities(train_features[mini_batch_indices], parameter_matrix)
            gradient_matrix = compute_gradient_matrix(train_features[mini_batch_indices], train_labels[mini_batch_indices], mini_batch_estimated_probabilities)
        elif method == 'stochastic':
            if stochastic_array is None or stochastic_loop_index == m:
                stochastic_array = get_random_sample_indices_no_replacement(m, m)[0]
                stochastic_loop_index = 0
            
            stochastic_index = [stochastic_array[stochastic_loop_index]]
            stochastic_estimated_probability = compute_softmax_estimated_probabilities(train_features[stochastic_index], parameter_matrix)
            gradient_matrix = compute_gradient_matrix(train_features[stochastic_index], train_labels[stochastic_index], stochastic_estimated_probability)
            stochastic_loop_index += 1
            

        # compute learning rate
        t += 1
        learning_rate = compute_learning_rate(t)

        # perform gradient step
        parameter_matrix = parameter_matrix - learning_rate * gradient_matrix

In [12]:
############################################ TEST ############################################

In [13]:
from sklearn import datasets

In [14]:
iris = datasets.load_iris()
features = iris['data']
m = np.shape(features)[0]
features_with_bias_column = np.hstack((np.full((m, 1), 1), features))

labels = (iris['target'] == 0).astype(int)
labels = np.vstack((labels, (iris['target'] == 1).astype(int)))
labels = np.vstack((labels, (iris['target'] == 2).astype(int)))
labels = labels.T

In [15]:
# split into training and test sets

In [16]:
test_set_size = math.ceil(.2 * m)

test_indices, train_indices = get_random_sample_indices_no_replacement(m, test_set_size)

train_features_with_bias_column = features_with_bias_column[train_indices]
train_labels = labels[train_indices]

test_features_with_bias_column = features_with_bias_column[test_indices]
test_labels = labels[test_indices]

In [17]:
# scale train and test features

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [19]:
n = np.shape(features_with_bias_column)[1]
scaler_that_ignores_bias_column = ColumnTransformer([('constant', 'passthrough', slice(1)), ('feature_weights', StandardScaler(), slice(1, n))])
train_features_scaled_linear = scaler_that_ignores_bias_column.fit_transform(train_features_with_bias_column)
test_features_scaled_linear = scaler_that_ignores_bias_column.transform(test_features_with_bias_column)

In [20]:
train_features_with_bias_column

array([[1. , 5.6, 2.7, 4.2, 1.3],
       [1. , 6.3, 3.3, 4.7, 1.6],
       [1. , 5.7, 2.9, 4.2, 1.3],
       [1. , 6.3, 2.9, 5.6, 1.8],
       [1. , 6.9, 3.1, 4.9, 1.5],
       [1. , 4.3, 3. , 1.1, 0.1],
       [1. , 5.8, 2.8, 5.1, 2.4],
       [1. , 5.2, 3.4, 1.4, 0.2],
       [1. , 7.9, 3.8, 6.4, 2. ],
       [1. , 5.4, 3.9, 1.7, 0.4],
       [1. , 5.6, 3. , 4.5, 1.5],
       [1. , 6.5, 3. , 5.5, 1.8],
       [1. , 4.9, 2.5, 4.5, 1.7],
       [1. , 7.7, 3. , 6.1, 2.3],
       [1. , 4.8, 3. , 1.4, 0.1],
       [1. , 5.5, 2.5, 4. , 1.3],
       [1. , 6.5, 2.8, 4.6, 1.5],
       [1. , 6.2, 2.2, 4.5, 1.5],
       [1. , 7.7, 2.8, 6.7, 2. ],
       [1. , 5.1, 3.8, 1.6, 0.2],
       [1. , 7. , 3.2, 4.7, 1.4],
       [1. , 5.8, 2.7, 5.1, 1.9],
       [1. , 6.7, 3.3, 5.7, 2.5],
       [1. , 4.8, 3. , 1.4, 0.3],
       [1. , 6.7, 3.1, 4.7, 1.5],
       [1. , 6.1, 2.8, 4.7, 1.2],
       [1. , 5. , 3.4, 1.5, 0.2],
       [1. , 5.7, 3.8, 1.7, 0.3],
       [1. , 6.4, 3.2, 4.5, 1.5],
       [1. , 6

In [21]:
train_features_scaled_linear

array([[ 1.        , -0.27026963, -0.80343997,  0.26748398,  0.12061433],
       [ 1.        ,  0.58001686,  0.55768186,  0.5535631 ,  0.51535212],
       [ 1.        , -0.14880014, -0.34973269,  0.26748398,  0.12061433],
       [ 1.        ,  0.58001686, -0.34973269,  1.06850553,  0.77851064],
       [ 1.        ,  1.30883385,  0.10397458,  0.66799475,  0.38377285],
       [ 1.        , -1.84937312, -0.12287905, -1.50620658, -1.45833684],
       [ 1.        , -0.02733064, -0.57658633,  0.7824264 ,  1.56798623],
       [ 1.        , -0.75614763,  0.7845355 , -1.33455911, -1.32675758],
       [ 1.        ,  2.52352884,  1.69195006,  1.52623212,  1.04166917],
       [ 1.        , -0.51320863,  1.9188037 , -1.16291164, -1.06359905],
       [ 1.        , -0.27026963, -0.12287905,  0.43913145,  0.38377285],
       [ 1.        ,  0.82295585, -0.12287905,  1.0112897 ,  0.77851064],
       [ 1.        , -1.12055613, -1.25714725,  0.43913145,  0.64693138],
       [ 1.        ,  2.28058984, -0.1

In [22]:
# train

In [23]:
best_parameter_matrix_linear, best_cost_linear, number_of_gradient_steps_performed_linear = softmax_regression_with_early_stopping(train_features_scaled_linear, train_labels, method='stochastic')

In [24]:
best_cost_linear

0.04167539501104463

In [25]:
best_parameter_matrix_linear

array([[-0.55483988, -1.80516521,  1.84409505, -3.33528908, -3.0285141 ],
       [ 3.48598318,  1.45531207, -0.64757287, -0.46197062, -1.82172403],
       [-2.9311433 ,  0.34985314, -1.19652218,  3.79725969,  4.85023813]])

In [26]:
number_of_gradient_steps_performed_linear

1080702

In [27]:
# train on degree 2 polynomial expansion of features

In [28]:
train_features = features[train_indices]
test_features = features[test_indices]

In [29]:
from sklearn.preprocessing import PolynomialFeatures
poly_degree_2 = PolynomialFeatures()

train_features_poly = poly_degree_2.fit_transform(train_features)
test_features_poly = poly_degree_2.transform(test_features)

In [30]:
train_features_poly[0]

array([ 1.  ,  5.6 ,  2.7 ,  4.2 ,  1.3 , 31.36, 15.12, 23.52,  7.28,
        7.29, 11.34,  3.51, 17.64,  5.46,  1.69])

In [31]:
n_poly = np.shape(train_features_poly)[1]
scaler_that_ignores_constant_column_poly = ColumnTransformer([('constant', 'passthrough', slice(1)), ('feature_weights', StandardScaler(), slice(1, n_poly))])
train_features_scaled_poly = scaler_that_ignores_constant_column_poly.fit_transform(train_features_poly)
test_features_scaled_poly = scaler_that_ignores_constant_column_poly.transform(test_features_poly)

In [32]:
train_features_scaled_poly[0]

array([ 1.        , -0.27026963, -0.80343997,  0.26748398,  0.12061433,
       -0.32724737, -0.7643554 ,  0.0426849 , -0.04923693, -0.8069326 ,
        0.04956151, -0.02703614,  0.05262896, -0.07010014, -0.18753652])

In [33]:
best_parameter_matrix_poly, best_cost_poly, number_of_gradient_steps_performed_poly = softmax_regression_with_early_stopping(train_features_scaled_poly, train_labels, method='stochastic')

In [34]:
best_cost_poly

0.04699108002655002

In [35]:
best_parameter_matrix_poly

array([[-0.88147686, -0.52449854,  1.06354937, -1.33010975, -1.18428954,
        -0.39928552,  0.38886031, -0.93741861, -0.86084922,  1.05850608,
        -1.0015434 , -0.92183354, -0.8339952 , -0.69307422, -0.5192302 ],
       [ 3.20344577,  0.74133506, -0.3241343 ,  0.44960858,  0.32243724,
         0.49437006,  0.34062787, -0.01457722, -0.07442966, -0.65384348,
         0.72007292,  0.49109542, -1.11201224, -1.24810223, -1.53383983],
       [-2.32196891, -0.21683652, -0.73941507,  0.88050117,  0.8618523 ,
        -0.09508454, -0.72948818,  0.95199582,  0.93527889, -0.40466261,
         0.28147048,  0.43073812,  1.94600744,  1.94117644,  2.05307003]])

In [36]:
number_of_gradient_steps_performed_poly

1080249

In [37]:
# train on interaction expansion of features

In [38]:
interactions = PolynomialFeatures(interaction_only=True)

train_features_interactions = interactions.fit_transform(train_features)
test_features_interactions = interactions.transform(test_features)

In [39]:
n_interactions = np.shape(train_features_interactions)[1]
scaler_that_ignores_constant_column_interactions = ColumnTransformer([('constant', 'passthrough', slice(1)), ('feature_weights', StandardScaler(), slice(1, n_interactions))])
train_features_scaled_interactions = scaler_that_ignores_constant_column_interactions.fit_transform(train_features_interactions)
test_features_scaled_interactions = scaler_that_ignores_constant_column_interactions.transform(test_features_interactions)

In [40]:
train_features_scaled_interactions[0]

array([ 1.        , -0.27026963, -0.80343997,  0.26748398,  0.12061433,
       -0.7643554 ,  0.0426849 , -0.04923693,  0.04956151, -0.02703614,
       -0.07010014])

In [41]:
best_parameter_matrix_interactions, best_cost_interactions, number_of_gradient_steps_performed_interactions = softmax_regression_with_early_stopping(train_features_scaled_interactions, train_labels, method='stochastic')

In [42]:
best_cost_interactions

0.002212424668078495

In [43]:
best_parameter_matrix_interactions

array([[-0.88875671, -0.66929574,  1.61545186, -1.6561378 , -1.48032666,
         0.6455912 , -1.20242135, -1.11077206, -1.22420664, -1.14699542,
        -0.92018954],
       [ 3.61813832,  1.16978398, -0.69292944,  0.24438293, -0.18522749,
         0.39784814, -0.16013694, -0.52152232,  0.52649634,  0.03285018,
        -2.11161373],
       [-2.72938161, -0.50048824, -0.92252242,  1.41175487,  1.66555416,
        -1.04343934,  1.3625583 ,  1.63229438,  0.6977103 ,  1.11414524,
         3.03180327]])

In [44]:
number_of_gradient_steps_performed_interactions

1080369

In [45]:
# select best model (model with the lowest minimized validation set cost)

In [46]:
best_cost = min(best_cost_linear, best_cost_poly, best_cost_interactions)
if best_cost == best_cost_linear:
    best_parameter_matrix = best_parameter_matrix_linear
    test_features_scaled = test_features_scaled_linear
elif best_cost == best_cost_poly:
    best_parameter_matrix = best_parameter_matrix_poly
    test_features_scaled = test_features_scaled_poly
else:
    best_parameter_matrix = best_parameter_matrix_interactions
    test_features_scaled = test_features_scaled_interactions

In [47]:
# compute cost of best model on test set

In [48]:
test_estimated_probabilities = compute_softmax_estimated_probabilities(test_features_scaled, best_parameter_matrix)
compute_cross_entropy_cost(test_labels, test_estimated_probabilities)

0.11746522716317315

In [49]:
# compute accuracy of best model on test set just for fun

In [50]:
test_predictions = np.argmax(test_estimated_probabilities, 1)
test_predictions

array([2, 1, 1, 0, 2, 2, 1, 2, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 1, 0, 2,
       2, 1, 0, 1, 1, 0, 1, 1], dtype=int64)

In [51]:
test_targets = iris['target'][test_indices]
test_targets

array([1, 1, 2, 0, 2, 2, 1, 2, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 1, 0, 2,
       2, 1, 0, 1, 1, 0, 1, 1])

In [52]:
test_predictions_equals_test_targets = test_predictions == test_targets
test_predictions_equals_test_targets

array([False,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [53]:
accuracy = np.average(test_predictions_equals_test_targets)
accuracy

0.9333333333333333

In [54]:
# evaluate cost and accuracy of zeroed parameter matrix just for fun

In [55]:
n = np.shape(train_features_with_bias_column)[1]
K = np.shape(train_labels)[1]
parameter_matrix_zeroed = np.zeros((K, n))

test_estimated_probabilities_zeroed = compute_softmax_estimated_probabilities(test_features_scaled_linear, parameter_matrix_zeroed)
compute_cross_entropy_cost(test_labels, test_estimated_probabilities_zeroed)

1.0986122886681098

In [56]:
test_predictions_zeroed = np.argmax(test_estimated_probabilities_zeroed, 1)
test_predictions_zeroed

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [57]:
test_predictions_zeroed_equals_test_targets = test_predictions_zeroed == test_targets
test_predictions_zeroed_equals_test_targets

array([False, False, False,  True, False, False, False, False, False,
        True, False,  True,  True, False, False,  True,  True,  True,
       False, False,  True, False, False, False,  True, False, False,
        True, False, False])

In [58]:
accuracy_zeroed = np.average(test_predictions_zeroed_equals_test_targets)
accuracy_zeroed

0.3333333333333333